In [1]:
import ast
import copy
import numpy as np
import os

In [2]:
import cc.atlas as atlas
import cc.cartography as cartography
import cc.publication as publication
import cc.utils as utils
import cc.tex as tex

In [3]:
import trove
import verdict

# Literature Review

This notebook contains a record of how I performed the literature review for this work.

# Parameters

In [4]:
config_fp = '/Users/zhafen/paper_repos/Hot-Accretion-in-FIRE/literature_review/literature_review.trove'
cp = trove.config_parser.ConfigParser( config_fp )
pm = trove.link_params_to_config(
    config_fp,
    variation = 'coronal_mixing_accretion',
)

In [5]:
atlas_dir = pm['root_data_dir']

In [6]:
topics = {}
for variation in cp.variations:
    topics[variation] = ast.literal_eval( cp.get( variation, 'publications' ) )

# Load Data

## Topics Data

In [7]:
topics_fp = os.path.join( pm['root_data_dir'], 'topics.json' )
topics_data = verdict.Dict.from_json( topics_fp, create_nonexisting=True )
if 'not_included' not in topics_data:
    topics_data['not_included'] = {}

## Zotero Atlas

In [8]:
# Larger atlas
a_zotero = atlas.Atlas( atlas_dir, load_bibtex=False, data_fp=pm['zotero_atlas_fp'] )

Loading saved atlas data.


0it [00:00, ?it/s]
100%|██████████| 1870/1870 [00:00<00:00, 17024.59it/s]


In [9]:
# Vectorization
vp_dict = a_zotero.vectorize( projection_fp=pm['zotero_projection_fp'] )
c_zotero = cartography.Cartographer( **vp_dict )

Vectorizing text...
Using saved vectorized text...


/Users/zhafen/repos/verdict/verdict.py:1404: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  if hasattr( a[first_element_index][0], 'decode' ):


## Large Atlas

In [10]:
# Larger atlas
a = atlas.Atlas( atlas_dir, load_bibtex=False, )

Loading saved atlas data.


0it [00:00, ?it/s]
100%|██████████| 16186/16186 [00:00<00:00, 25996.39it/s]


In [11]:
# Vectorization
vp_dict = a.vectorize()
c = cartography.Cartographer( **vp_dict )

Vectorizing text...
Using saved vectorized text...


## Tex Draft

In [12]:
pub_doc = tex.Tex( filepath=pm['tex_fp'] )

# Create a Reading List

## Setup

In [13]:
if pm['variation'] not in topics_data['not_included']:
    topics_data['not_included'][pm['variation']] = []

In [14]:
def sort_by_similarity( target_key, c ):
    
    cospsi = c.cospsi( target_key, 'all' )
    sort_inds = np.argsort(cospsi)[::-1]
    sorted_cospsi = cospsi[sort_inds]
    sorted_publications = c.publications[sort_inds]
    
    return sorted_cospsi, sorted_publications

In [29]:
def print_sorted_publications(
    sorted_cospsi,
    sorted_publications,
    kernel_size,
    a,
    show_unread_only = False,
    do_not_show_included = True,
    do_not_show_not_included = True,
    central_publication = '',
):

#     print( 'The {} most related publications are...\n'.format( kernel_size ) )

    n_shown = 0
    for i, key_i in enumerate( sorted_publications ):
                
        if i > kernel_size:
            break
                
        if key_i not in a.data:
            continue
        
        p_i = a[key_i]

        if 'read' in p_i.notes:
            read_flag = p_i.notes['read']
        else:
            read_flag = 'UNREAD'
        if show_unread_only and read_flag != 'UNREAD':
            continue
            
        included_flag = key_i in pub_doc.string           
        deliberately_not_included = key_i in topics_data['not_included'][pm['variation']]
        
        if do_not_show_included and included_flag:
            continue
        if do_not_show_not_included and deliberately_not_included:
            continue

        if not isinstance( p_i, publication.UnofficialPublication ):
            print( '{} -- {}'.format( n_shown, key_i, ) )
            print( p_i.citation['title'] )
            print( p_i.citation['author'] )
            print( 'Related rank: {}.{}'.format( i, central_publication ) ) 
            print( 'Read: {}'.format( ''.join( read_flag ) ) )
            print( 'Included: {}'.format( included_flag ) )
            print( p_i.citation['ENTRYTYPE'] + '\n' )
            print( p_i.points_str() + '\n\n' )
        else:
            print( '{} -- {}'.format( n_shown, key_i, ) )
            print( 'Related rank: {}.{}'.format( i, central_publication ) ) 
        
        n_shown += 1



## Conservative Reading List
For the average abstract in combination with chosen words.

### Create and add average vector

In [16]:
# Can add chosen words if so wished.
search_str = ''

In [17]:
for i, key_i in enumerate( pm['publications'] ):
    search_str += a[key_i].points_str()

In [18]:
p = publication.UnofficialPublication( pm['variation'] )

In [19]:
p.process_abstract( abstract_str=search_str )

In [20]:
a.data[pm['variation']] = p

In [22]:
a.update( a_zotero )
a.prune_duplicates(preferred=list(a_zotero.data.keys()))

100%|██████████| 13320/13320 [00:00<00:00, 116316.57it/s]


In [23]:
# Vectorize
vp_dict = a.vectorize( overwrite=True, projection_fp='pass', )
c = cartography.Cartographer( **vp_dict )

Vectorizing text...
    Retrieving publication data...


  0%|          | 0/15103 [00:00<?, ?it/s]/Users/zhafen/repos/cc/cc/publication.py:800: UserWarning: Hafen2021 is unofficial and has no publication date.
  warnings.warn( warning_msg )
/Users/zhafen/repos/cc/cc/publication.py:800: UserWarning: coronal_mixing_accretion is unofficial and has no publication date.
  warnings.warn( warning_msg )
100%|██████████| 15103/15103 [00:00<00:00, 54512.59it/s]


    Calculating vectorization...


### Produce reading list

In [25]:
sorted_cospsi, sorted_publications = sort_by_similarity( pm['variation'], c )

In [64]:
pub_doc = tex.Tex( filepath=pm['tex_fp'] )

In [78]:
topics_data['not_included'][pm['variation']].append( 'Sancisi2008' )
topics_data.to_json( topics_fp )

In [81]:
print_sorted_publications(
    sorted_cospsi,
    sorted_publications,
    pm['kernel_size'] * 2,
    a,
    do_not_show_included = True,
    do_not_show_not_included = True,
    central_publication = pm['variation'],
)

## Extensive Reading List
For each and everyone of the publications.

In [ ]:
a_pubs = []
sorted_cospsis = []
sorted_publications = []
for i, key_i in enumerate( pm['publications'] ):
    a_i = atlas.Atlas(atlas_dir, load_bibtex=False, load_atlas_data=False )
    a_i.data[key_i] = a[key_i]
    
    # Identify relevant publications
    sorted_cospsi_i, sorted_publications_i = sort_by_similarity( key_i, c )
    sorted_cospsis.append( sorted_cospsi_i )
    sorted_publications.append( sorted_publications_i )

    for key in sorted_publications_i:

        # Get missing publications
        if key in a_zotero.data:
            a_i.data[key] = a_zotero.data[key]
        elif key in a.data:
            a_i.data[key] = a.data[key]

    a_i.prune_duplicates(preferred=list(a_zotero.data.keys()))
    
    a_pubs.append( a_i )

In [ ]:
pub_doc = tex.Tex( filepath=pm['tex_fp'] )

In [ ]:
topics_data['not_included'][pm['variation']].append( '2017MNRAS.466.3460V' )
topics_data.to_json( topics_fp )

In [ ]:
for i, a_i in enumerate( a_pubs ):
    
#     print( '===============================================================================' )
#     print( pm['publications'][i] + '\n' )
    
    print_sorted_publications(
        sorted_cospsis[i],
        sorted_publications[i],
        pm['kernel_size'],
        a_i,
        do_not_show_included = True,
        do_not_show_not_included = True,
        central_publication = pm['publications'][i],
    )